Once in interactive mode, use the mouse to click or highlight a block. Then use the "run" button on top or "shift + enter" to execute the block.

To make a new block click on one of the horizontal bars. Here you can add Sage code or type in comments.

In the following worksheet we demonstrate how to compute the class group of a quadratic field as well as decompose ideals over a set of generators for the class group.

In [1]:
# In the following, consider the quadratic field with fundamental discriminant -1363.
D = -1363
K = QuadraticField(D)
K

Number Field in a with defining polynomial x^2 + 1363 with a = 36.91882988394947?*I

In [2]:
# Ideal classes of O_K can be represented with reduced quadratic forms of discriminant -1363.

f = BinaryQF([7, 3, 49])
f.is_reduced()
f.discriminant()    # verify the form has the right discriminant

-1363

In [3]:
# Composition of forms is denoted by multiplication, but the result is not necessarily reduced.

f*f
(f*f).reduced_form()

7*x^2 - 3*x*y + 49*y^2

In [4]:
# If f = (a, b, c) is a reduced form, then its inverse is (a, -b, c)

g = BinaryQF([7, -3, 49])
g.is_reduced()

I = f*g
I.reduced_form()

x^2 + x*y + 341*y^2

In [5]:
# The split prime ideals of K with norm less than Minkowski's bound generate the class group of K.
B = floor(K.minkowski_bound())
B

23

Q1. What are the ramified primes of K? Why do we not need to consider them when computing the class group?

They are $29$ and $47$ because $x^2+1363\bmod{29}$ and $x^2+1363\bmod{47}$ are equivalent to $x^2$, which has roots of multiplicity > 1. We cannot generate the prime ideals of a class group efficiently with ramified primes. It is possible that ramified primes generate only one prime ideal but it is guaranteed that split primes generate at least two ideals. Lecture 2 Example 1 shows how to generate prime ideals.

Q2. What is the ideal class of an inert prime? Why do we not need to consider them when computing the class group?

They are the set of integers $y$ satisfying "$x^2+1363\bmod{y}$ is irreducible" for $y\in\mathbb{Z}_{p}$. We cannot generate the prime ideals of a class group efficiently with inert primes. Inert primes generate one prime ideal only.

In [6]:
P = Primes()
p = P.first()
ramifieds = []
inerts = []
splits = []
while p < 1363:
    if 1363 % p == 0:
        ramifieds.append(p)
    else:
        R.<x> = GF(p)[]
        f = x**2 + 1363
        if f.is_irreducible() or p == 2: # (x^2+1363) mod 2 = (x^2+1) mod 2 = (x+1)^2 mod 2. without mod 2, is irreducible
            inerts.append(p)
        else:
            splits.append(p)
    p = P.next(p)

# check work
# https://ask.sagemath.org/question/8349/splitting-of-primes/
D = -1363
K = QuadraticField(D)
ramifieds1 = [x[0] for x in list(K.discriminant().factor())]
is_split = lambda F, x: sum([t[1] for t in list(F.factor(x))]) > 1
splits1 = []
inerts1 = []
for x in range(1363):
    if x in ramifieds1:
        continue
    if is_prime(x):
        if is_split(K, x):
            splits1.append(x)
        else:
            inerts1.append(x)
assert ramifieds == ramifieds1
assert splits == splits1
assert inerts == inerts1

In [7]:
# We can construct the forms corresponding to reduced prime ideals. This is the prime form above 7, with its conjugate given by its inverse.

p = 7
R = Integers(2*p)

a = p
b = int(sqrt(R(D)))        # square root of D mod 7
c = (b^2 - D)/(4*a)

f = BinaryQF([a, b, c])
print(a, b, c)
f = f.reduced_form()
f

f.discriminant()

7 3 49


-1363

Q4. Find the binary quadratic forms f1, f2, ..., fn corresponding to the split prime ideals with norm less than B.

According to the below calculations, the binary quadratic forms are:

1. $7*x^2 + 3*x*y + 49*y^2$
2. $11*x^2 + x*y + 31*y^2$
3. $19*x^2 + 9*x*y + 19*y^2$

In [8]:
reduced_splits = [split_ for split_ in splits if split_ < B]
print(reduced_splits)

def get_quadratic_form(p):
    R = Integers(2*p)

    a = p
    b = int(sqrt(R(D)))
    c = (b^2 - D)/(4*a)

    return a, b, c

f1 = BinaryQF(get_quadratic_form(7)).reduced_form()
f2 = BinaryQF(get_quadratic_form(11)).reduced_form()
f3 = BinaryQF(get_quadratic_form(19)).reduced_form()

print(f1)
print(f2)
print(f3)

[7, 11, 19]
7*x^2 + 3*x*y + 49*y^2
11*x^2 + x*y + 31*y^2
19*x^2 + 9*x*y + 19*y^2


In [9]:
# We now need relations of the form f1^x1 * f2^x2 * ... * fn^xn = I = (1, 1, 341).
# Since exponentiation of forms is not natively supported here is a (not efficient) function to help:

id = BinaryQF([1, 1, 341])

def inv(f):
    g = f.reduced_form()
    return BinaryQF([g[0], -g[1], g[2]]).reduced_form()

def ex(f, n):
    g = id
    for i in range(abs(n)):
        g *= f
    g = g.reduced_form()
    if n < 0:
        g = inv(g)
    return g

inv(f)
ex(f, 2) == (f * f).reduced_form()

True

Q5. Write a function that generates n random numbers in [-10, 10]. Look for relations of the form f1^x1 * f2^x2 * ... * fn^xn = I = (1, 1, 341).

One such relation that satisfies this is $x_1=-5, x_2=2, x_3=3$.

In [10]:
# Here is a simple example of working with matrices in Sage, especially adding rows to a matrix:

M = matrix([[1, 2, 4], [5, 4, 1], [2, 4, 3]])

rows = M.rows()
rows += [(5, 6, 7)]

matrix(rows)

[1 2 4]
[5 4 1]
[2 4 3]
[5 6 7]

In [11]:
while True:
    x1 = randint(-10, 10)
    x2 = randint(-10, 10)
    x3 = randint(-10, 10)
    if ex(f1, x1) * ex(f2, x2) * ex(f3, x3) == id:
        print(x1, x2, x3)
        break

-9 6 8


Q6. Construct the matrix with row vectors given by (x1, x2, ..., xn). Continue searching for relations as in Q5 until the matrix has full rank.

See matrix result below

In [12]:
m = []

# don't trust M.rank() == 3 or M.rank().transpose() == 3 or M.determinant() != 0
# the hermite form would still differ after running twice
while len(m) <= 12:
    x1 = randint(-10, 10)
    x2 = randint(-10, 10)
    x3 = randint(-10, 10)
    if ex(f1, x1) * ex(f2, x2) * ex(f3, x3) == id:
        m.append([x1, x2, x3])
        M = matrix(m)

print(M)

[ -4   4   2]
[  9   0  -4]
[  4 -10   6]
[  8  10 -10]
[  4   2   4]
[  7   8  -6]
[  9   6   4]
[  6   0   4]
[-10  -8 -10]
[  6   6   2]
[  2   4   4]
[ -4   4  -6]
[ -6  -6   2]


In [13]:
# Recall that the Hermite normal form of this matrix gives the structure of the class group of K.
# We can create the (upper triangular!) HNF by:

M = M.hermite_form()
print(M)

# and the Smith normal form by:

H, U, V = M.smith_form()

[1 2 0]
[0 6 0]
[0 0 2]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]


Q7. What is the structure of the class group? (Hint: the class number is less than 12. If you still aren't there, try adding more relations.)

The structure is
	$$
	\begin{bmatrix} 
	1 & 2 & 0 \\
	0 & 6 & 0 \\
	0 & 0 & 2 \\
	\end{bmatrix}
	\quad
	$$

Q8. Find elements of each possible order in the class group (hint: see page 3 of lecture 7).

The elements of each possible order in the group are

1. $x^2 + x*y + 341*y^2$
2. $19*x^2 + 9*x*y + 19*y^2$
3. $7*x^2 - 3*x*y + 49*y^2$

Q9. Decompose the quadratic form (7, 3, 49) over the generators of the class group (hint: can be solved by inspection).

One example is ${g_0}^0 * {g_1}^0 * {g_2}^1$

Q10. The class group is cyclic, so from Q8 we should have one generator of order equal to the class number, call it g. For the prime forms f above 7, 11, and 19, solve the discrete log problem f = g^x.

7 and 11 don't have a solution. For 19, $x = 3$.

In [14]:
generators = V^-1
g0 = ex(f1, generators[0][0]) * ex(f2, generators[0][1]) * ex(f3, generators[0][2])
g1 = ex(f1, generators[1][0]) * ex(f2, generators[1][1]) * ex(f3, generators[1][2])
g2 = ex(f1, generators[2][0]) * ex(f2, generators[2][1]) * ex(f3, generators[2][2])
g0 = g0.reduced_form()
g1 = g1.reduced_form()
g2 = g2.reduced_form()
print(g0)
print(g1)
print(g2)

x^2 + x*y + 341*y^2
19*x^2 + 9*x*y + 19*y^2
7*x^2 - 3*x*y + 49*y^2


In [15]:
assert ex(g1, 3) == g1

stopf1 = False
stopf2 = False
stopf3 = False
for i in range(2, 11):
    if ex(g1, i) == f1 and not stopf1:
        print("f1", i)
        stopf1 = True
    elif ex(g1, i) == f2 and not stopf2:
        print("f2", i)
        stopf2 = True
    elif ex(g1, i) == f3 and not stopf3:
        print("f3", i)
        stopf3 = True

f3 3
